<a href="https://colab.research.google.com/github/bkvkrll/Introduction-to-Neural-Networks/blob/main/lesson3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Практическое задание**

Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.


Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?


Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).


1-2. (*) Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [1]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
!pip install tensorflow -latest


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -l


In [3]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: gast
    Found existing installatio

In [4]:
!pip install mnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Convolution1D, LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

import tensorflow_datasets as tfds

import mnist
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

print(tf.__version__)

2.9.1


In [7]:
boston_data = load_boston()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [8]:
df = pd.DataFrame(data=boston_data.data, columns = boston_data.feature_names)

In [9]:
df['target'] = boston_data.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [10]:
d = preprocessing.normalize(df.drop(columns='target'))

scaled_df = pd.DataFrame(d, columns=boston_data.feature_names)
scaled_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000013,0.035997,0.004620,0.0,0.001076,0.013149,0.130388,0.008179,0.002000,0.591945,0.030597,0.793727,0.009959
1,0.000058,0.000000,0.014977,0.0,0.000994,0.013602,0.167140,0.010522,0.004237,0.512648,0.037707,0.840785,0.019362
2,0.000059,0.000000,0.015174,0.0,0.001007,0.015421,0.131140,0.010661,0.004293,0.519409,0.038204,0.843138,0.008650
3,0.000071,0.000000,0.004785,0.0,0.001005,0.015360,0.100527,0.013306,0.006585,0.487268,0.041045,0.866174,0.006453
4,0.000151,0.000000,0.004755,0.0,0.000999,0.015587,0.118209,0.013222,0.006543,0.484177,0.040784,0.865631,0.011625


In [11]:
scaled_df['target'] = df['target']
scaled_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.000013,0.035997,0.004620,0.0,0.001076,0.013149,0.130388,0.008179,0.002000,0.591945,0.030597,0.793727,0.009959,24.0
1,0.000058,0.000000,0.014977,0.0,0.000994,0.013602,0.167140,0.010522,0.004237,0.512648,0.037707,0.840785,0.019362,21.6
2,0.000059,0.000000,0.015174,0.0,0.001007,0.015421,0.131140,0.010661,0.004293,0.519409,0.038204,0.843138,0.008650,34.7
3,0.000071,0.000000,0.004785,0.0,0.001005,0.015360,0.100527,0.013306,0.006585,0.487268,0.041045,0.866174,0.006453,33.4
4,0.000151,0.000000,0.004755,0.0,0.000999,0.015587,0.118209,0.013222,0.006543,0.484177,0.040784,0.865631,0.011625,36.2


In [12]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df.drop(columns='target'), scaled_df['target'], test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((354, 13), (152, 13), (354,), (152,))

In [13]:
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

In [14]:
results = {}

In [15]:
def model_b(layers_count, neurons_count, opt):
    for i in range (0,layers_count-1):
        neurons=neurons_count
        md = Sequential([tf.keras.layers.Dense(neurons, activation=tf.nn.relu)])
    md.add(Dense(1, activation='relu'))
    md.compile(optimizer=opt, loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
    md.fit(X_train, y_train, batch_size=8, validation_split=0.20, epochs=20)
    loss_mae = md.evaluate(X_test, y_test)
    results['model' + '_' + str(layers) + '_' + str(neurons)+ '_' + opt] = loss_mae[1]
    print(f"Слоев: {layers_count}, нейронов в слое: {neurons}")
    return md

In [16]:
for layers in (2, 5):
    for neurons in (64, 128, 256, 512):
        for opt in ('adam', 'rmsprop', 'nadam'):
            print(layers, neurons, opt)
            model_b(layers, neurons, opt)

2 64 adam
Epoch 1/20
36/36 [==============================] - 1s 8ms/step - loss: 22.5299 - mae: 22.5299 - val_loss: 21.8330 - val_mae: 21.8330
Epoch 2/20
36/36 [==============================] - 0s 4ms/step - loss: 21.5786 - mae: 21.5786 - val_loss: 20.7615 - val_mae: 20.7615
Epoch 3/20
36/36 [==============================] - 0s 3ms/step - loss: 20.1899 - mae: 20.1899 - val_loss: 19.2013 - val_mae: 19.2013
Epoch 4/20
36/36 [==============================] - 0s 3ms/step - loss: 18.4920 - mae: 18.4920 - val_loss: 17.4816 - val_mae: 17.4816
Epoch 5/20
36/36 [==============================] - 0s 3ms/step - loss: 16.7203 - mae: 16.7203 - val_loss: 15.5551 - val_mae: 15.5551
Epoch 6/20
36/36 [==============================] - 0s 3ms/step - loss: 14.8056 - mae: 14.8056 - val_loss: 13.3720 - val_mae: 13.3720
Epoch 7/20
36/36 [==============================] - 0s 4ms/step - loss: 12.5013 - mae: 12.5013 - val_loss: 10.8039 - val_mae: 10.8039
Epoch 8/20
36/36 [==============================] - 

In [17]:
results

{'model_2_128_adam': 3.0928890705108643,
 'model_2_128_nadam': 3.1496822834014893,
 'model_2_128_rmsprop': 3.062009572982788,
 'model_2_256_adam': 3.024937391281128,
 'model_2_256_nadam': 2.9664504528045654,
 'model_2_256_rmsprop': 2.915403366088867,
 'model_2_512_adam': 2.9723823070526123,
 'model_2_512_nadam': 3.0349836349487305,
 'model_2_512_rmsprop': 2.73755145072937,
 'model_2_64_adam': 3.453705310821533,
 'model_2_64_nadam': 3.4048991203308105,
 'model_2_64_rmsprop': 3.7656662464141846,
 'model_5_128_adam': 3.2026500701904297,
 'model_5_128_nadam': 3.099148750305176,
 'model_5_128_rmsprop': 3.3024280071258545,
 'model_5_256_adam': 2.9800736904144287,
 'model_5_256_nadam': 3.051827907562256,
 'model_5_256_rmsprop': 2.949784517288208,
 'model_5_512_adam': 2.9292850494384766,
 'model_5_512_nadam': 2.8815665245056152,
 'model_5_512_rmsprop': 3.3150370121002197,
 'model_5_64_adam': 3.374372720718384,
 'model_5_64_nadam': 3.348426342010498,
 'model_5_64_rmsprop': 3.484327554702759}

In [18]:
res = pd.DataFrame(results.items(), columns=['Name', 'MAE'])
res

,Name,MAE
0,model_2_64_adam,3.453705
1,model_2_64_rmsprop,3.765666
2,model_2_64_nadam,3.404899
3,model_2_128_adam,3.092889
4,model_2_128_rmsprop,3.062010
5,model_2_128_nadam,3.149682
6,model_2_256_adam,3.024937
7,model_2_256_rmsprop,2.915403
8,model_2_256_nadam,2.966450
9,model_2_512_adam,2.972382
